In [4]:
import os

import numpy as np
import pandas as pd

from data_loader import label_str2int_dict

rationale_names = {'td','itc'}
current_path = os.getcwd()
print(current_path)
os.chdir('/home/lyq/DataSet/FakeNews/twitter')
print(os.getcwd())



/home/lyq/DataSet/FakeNews/twitter
/home/lyq/DataSet/FakeNews/twitter


In [5]:
df = pd.read_csv('twitter.csv',encoding='utf-8')
df.rename(columns={'post_id':'source_id','post_text':'content'},inplace=True)
# content,label,image_id,source_id,split,td_rationale,td_pred,td_acc,cs_rationale,cs_pred,cs_acc
df['label'] = df['label'].apply(lambda x: label_str2int_dict[x])
df['split'] = np.nan
df = df[['source_id','content','label','split','image_id']]
df.columns

Index(['source_id', 'content', 'label', 'split', 'image_id'], dtype='object')

In [6]:
for rationale_name in rationale_names:
    rationale_df = pd.read_csv(f'{rationale_name}.csv')
    rationale_df = rationale_df.dropna(subset=['authenticity','reason'])
    df = df.merge(rationale_df,on='source_id',how='left')
    df = df.rename(columns={
        'authenticity':f'{rationale_name}_pred',
        'reason': f'{rationale_name}_rationale',
    })
    df = df.dropna(subset=[f'{rationale_name}_pred',f'{rationale_name}_rationale'])
    df[f'{rationale_name}_pred'] = df[f'{rationale_name}_pred'].apply(lambda x: label_str2int_dict[x])
    df[f'{rationale_name}_acc'] = df.apply(lambda x : int(x[f'{rationale_name}_pred']==x['label']) ,axis=1)
    print(df.columns)
  
df
    

Index(['source_id', 'content', 'label', 'split', 'image_id', 'td_pred',
       'td_rationale', 'td_acc'],
      dtype='object')
Index(['source_id', 'content', 'label', 'split', 'image_id', 'td_pred',
       'td_rationale', 'td_acc', 'itc_pred', 'itc_rationale', 'itc_acc'],
      dtype='object')


,source_id,content,label,split,image_id,td_pred,td_rationale,td_acc,itc_pred,itc_rationale,itc_acc
0,325145334739267584,PIC: Comparison of #Boston suspect Sunil Tripa...,1,NaN,boston_fake_23,1,1. The message suggests a comparison between ...,1,0,The image contains a missing person poster fo...,0
1,325152091423248385,I'm not completely convinced that it's this Su...,1,NaN,boston_fake_34,1,1. The message is a tweet expressing doubt ab...,1,0,"The image shows two individuals, one with a h...",0
2,324315545572896768,4chan and the bombing. just throwing it out th...,1,NaN,boston_fake_15,1,1. The message is extremely brief and lacks c...,1,0,The image shows before and after scenes of a ...,0
3,324581777614180352,4chan thinks they found pictures of the bomber...,1,NaN,boston_fake_08,1,1. Source: The source of the message is 4chan...,1,0,The image shows a crowded scene with individu...,0
4,324665423956176896,"Ola ke ase, investigando las bombas de Boston ...",1,NaN,boston_fake_35,1,1. Language: The message is written in a mix ...,1,0,The image shows a crowd with individuals high...,0
...,...,...,...,...,...,...,...,...,...,...,...
14190,700662094107119616,14 children! 14 different fathers! All of them...,1,NaN,woman_14_children_1,1,1. The claim that a woman has 14 children wit...,1,0,"The image shows a couple with a newborn baby,...",0
14191,700638809155710977,Meet The Woman Who Has Given Birth To 14 Child...,1,NaN,woman_14_children_1,1,1. Sensationalism: The headline is highly sen...,1,0,The image shows a woman holding a newborn bab...,0
14192,700618091886047232,"RT @Viasat1Ghana: Woman, 36, gives birth to 14...",1,NaN,woman_14_children_1,1,1. **Source**: The source of the tweet is Via...,1,0,The image shows a woman in hospital attire ho...,0
14193,700569502874992640,Woman Breaks World Record With 14 Children fro...,1,NaN,woman_14_children_2,1,1. **Source**: The source of the message is n...,1,0,"The image shows a diverse group of children, ...",0


In [7]:
print(f"before filtering : {df.shape[0]}  real data :{(df['label']==1).sum()} fake data: {(df['label']==0).sum()}")

def valid_row(row):
    condition = [(row['label'] in [0,1]),(row['content'] is not None and len(row['content'])>0)]
    for rationale_name in rationale_names:
        condition.extend(
            [(row[f'{rationale_name}_pred'] in [0,1,2]),
             (row[f'{rationale_name}_acc'] in [0,1]),
             (row[f'{rationale_name}_rationale'] is not None and len(row[f'{rationale_name}_rationale']) > 0)]
        )
    return all(condition)

df = df[df.apply(valid_row,axis=1)]
print(f"after filtering : {df.shape[0]}  real data :{(df['label']==1).sum()} fake data: {(df['label']==0).sum()}")

before filtering : 14195  real data :7763 fake data: 6432
after filtering : 14195  real data :7763 fake data: 6432


In [8]:
from Util import cal_rationale_metrics
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix, accuracy_score

for r_name in rationale_names:
    pred = df[r_name+ "_pred"].tolist()
    label = df['label'].tolist()
    print(cal_rationale_metrics(pred,label))
    

{'acc': 0.6865797816132441, 'recall': 0.6794081128878344, 'recall_real': 0.6029228855721394, 'recall_fake': 0.7558933402035296, 'precision': 0.6842472037243117, 'precision_real': 0.671747791442924, 'precision_fake': 0.6967466160056993, 'f1_macro': 0.6802965557120234, 'f1_real': 0.6354772634166326, 'f1_fake': 0.7251158480074142}
{'acc': 0.5533638605142656, 'recall': 0.5907064073391897, 'recall_real': 0.9889614427860697, 'recall_fake': 0.19245137189230968, 'precision': 0.7291373548956666, 'precision_real': 0.5036421219319082, 'precision_fake': 0.954632587859425, 'f1_macro': 0.49386350633744935, 'f1_real': 0.6674011121603189, 'f1_fake': 0.32032590051457976}


In [9]:
from sklearn.model_selection import train_test_split


def split_data(df, train_ratio=0.8, valid_ratio=0.1, random_state=None, stratify_col=None):
    """
    将 pandas DataFrame 划分为训练集、验证集和测试集。

    参数:
        df (pd.DataFrame): 包含所有数据的 pandas DataFrame。
        train_ratio (float): 训练集所占比例，默认为 0.8。
        valid_ratio (float): 验证集所占比例，默认为 0.1。
        random_state (int, RandomState instance or None): 控制随机抽样的种子，默认为 None。
        stratify_col (str or None): 如果不为 None，则数据将按照指定列进行分层抽样，默认为 None。

    返回:
        tuple: 包含训练集、验证集和测试集的元组。
               每个集合都是原始 DataFrame 的一部分，包含所有原始列。
    """
    # 计算测试集的比例
    test_ratio = 1 - train_ratio - valid_ratio
    if test_ratio <= 0:
        raise ValueError("The sum of train_ratio and valid_ratio must be less than 1.")
    
    # 分层抽样的依据列（如果指定了）
    stratify = df[stratify_col] if stratify_col else None
    
    # 第一步：先从全部数据中划分出测试集
    remaining_df, test_df = train_test_split(
        df, 
        test_size=test_ratio , 
        random_state=random_state, 
        shuffle=True,
        stratify=stratify
    )
    
    # 更新分层抽样的依据列（如果指定了）
    stratify_remaining = remaining_df[stratify_col] if stratify_col else None
    
    # 第二步：从剩余的数据中划分出训练集和验证集
    train_df, valid_df = train_test_split(
        remaining_df,
        test_size=valid_ratio / (train_ratio + valid_ratio),
        random_state=random_state,
        shuffle=True,
        stratify=stratify_remaining
    )

    return train_df, valid_df, test_df


train_df,valid_df,test_df = split_data(df,train_ratio=0.8,valid_ratio=0.1,random_state=666,stratify_col='label')


train_df.shape[0],valid_df.shape[0],test_df.shape[0]

(11355, 1420, 1420)

In [10]:

df.to_csv('twitter_llm_rationales.csv',index=False)
train_df.to_csv('train.csv',index=False)
valid_df.to_csv('val.csv',index=False)
test_df.to_csv('test.csv',index=False)